In [2]:
import spacy
import pandas as pd
from spacy import displacy
from spacy.matcher import Matcher as mt

In [ ]:
nlp = spacy.load('en_core_web_sm')
#print(nlp.pipe_names)

In [49]:
def hasProgLang (doc):
    for i in doc:
        if i.lower_ in ['go', 'golang', 'python', 'objective-c']:
            if i.pos_ != 'VERB':
                return True
    return False

In [53]:
doc = nlp('I am learning how to program in objective-c')
hasProgLang(doc)

False

In [54]:
[t for t in doc]

[I, am, learning, how, to, program, in, objective, -, c]

In [ ]:
displacy.render(doc)

<h1>Using Matcher</h1>

In [55]:
matcher = mt(nlp.vocab)

pattern_objc = [{'LOWER': 'objective'}, {'IS_PUNCT': True, 'OP' : '?'}, {'LOWER': 'c'}, ]
pattern_objc2 = [{'LOWER': 'objectivec'}]

matcher.add('objc', [pattern_objc])
matcher.add('objc2', [pattern_objc2])

<img src = 'img/opDescrip.PNG'>

In [57]:
doc = nlp('I am learning how to program in OBJECTIVE-C, objectivec')
matcher(doc)

[(1511212164044119274, 7, 10), (9524823172730451013, 11, 12)]

In [59]:
matcher.get(9524823172730451013)

(None, [[{'LOWER': 'objectivec'}]])

In [61]:
doc[11:]

objectivec

In [62]:
pattern_go = [{'LOWER': 
                   {'IN': ['go', 'golang']}, 
               'POS': 
                   {'NOT_IN': ['VERB']}
              }]
matcher.add('go', [pattern_go])
doc = nlp('I am learning how to program in objective-c as well in Go')
matcher(doc)

[(1511212164044119274, 7, 10), (8004577259940138793, 13, 14)]

In [63]:
#print(doc[7:10])
#print(doc[13:14])
for match_id, start, end in matcher(doc):
    print(doc[start:end])

objective-c
Go


In [76]:
pattern_objc = [{'LOWER': 'objective'}, {'IS_PUNCT': True, 'OP' : '?'}, {'LOWER': 'c'}, {'IS_PUNCT' : True, 'OP': '*'}]
pattern_go = [{'LOWER': 'go'}, {'POS': {'NOT_IN': ['VERB']}}]
pattern_python = [{'LOWER': 'python'}]
pattern_js = [{'LOWER': {'IN': ['js', 'javascript']}}]

matcher.add('objc', [pattern_objc])
matcher.add('go', [pattern_go])
matcher.add('python', [pattern_python])
matcher.add('js', [pattern_js])

In [66]:
doc = nlp('I hope I have mastered Python, Objective C and JS in a few years')
for match_id, start, end in matcher(doc):
    print(doc[start:end])

Python
Objective C
JS


In [68]:
doc = nlp('Go is a programming language')
matcher(doc)
doc[2:3]

a

In [69]:
[(i, i.pos_) for i in doc[:3]]

[(Go, 'VERB'), (is, 'AUX'), (a, 'DET')]

<h1>Benchmarking</h1>

In [70]:
df = (pd.read_csv("Questions.csv", nrows=1_000_000, encoding="ISO-8859-1", usecols=['Title', 'Id']))

In [77]:
titles = (_ for _ in df['Title'] if "objective-c" in _.lower())

In [78]:
for i in range(200):
    doc = nlp(next(titles))
    if len(matcher(doc)) == 0:
        print(doc)

__OBJC__ equivalent for Objective-C++
iPhone Device/Simulator memory oddities using Objective-C++
Objective-C. I have typedef float DuglaType[3]. How do I declare the property for this?
Learning Objective-C. Using Xcode 3.2.1. What is error: Program received signal: âEXC_ARITHMETICâ


In [81]:
doc = nlp('__OBJC__ equivalent for Objective-C++')
[t for t in doc]

[_, _, OBJC, _, _, equivalent, for, Objective, -, C++]